# Why do we choose the exponential function $e^{aq+b}+c$

We opt for the exponential family based on two observations regarding the correlation between 
$\varepsilon$ and $q$:
1. the theoretical result (see Eq.(8)) involves a finite number of logarithmic and exponential functions;
2. empirical findings (see Fig.5) suggest the existence of a convex relationship.

In addition to $e^{aq+b}+c$, we also have assessed two variants and found that increased complexity in the expression leads to marginal improvement in $R^2$. Following Occam's razor principle, we adopt $e^{aq+b}+c$ as our fit function. 

Nevertheless, it is essential to explore superior alternatives for more **substantial** improvements in the future.

In [1]:
import argparse
import copy
import datetime
import importlib
import numpy as np
import os
import pandas as pd
import time
import warnings # ignore warnings for clarity
warnings.simplefilter("ignore")

import torch
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST

from myopacus import PrivacyEngine
from myopacus.accountants.pdp_utils import GENERATE_EPSILONS_FUNC
from strategies import FedAvg, PFA

from datasets.fed_mnist import (
    FedClass, 
    RawClass, 
    BaselineModel,
    BaselineLoss,
    Optimizer,
    metric
)

NUM_LABELS = 10
BATCH_SIZE = 512
NUM_STEPS = 100
LR = 0.01
TARGET_DELTA = 0.0001
MAX_GRAD_NORM = 1.0
MAX_PHYSICAL_BATCH_SIZE = 512

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
project_abspath = os.path.abspath(os.path.join(os.getcwd(),".."))
data_path = os.path.join(project_abspath, "datasets/fed_mnist/iid_10")
rawdata = RawClass(data_path=data_path)
train_dataloader = FedClass(rawdata=rawdata, train=True, pooled=True)
test_dataloader = FedClass(rawdata=rawdata, train=False, pooled=True)
print(len(train_dataloader), len(test_dataloader))

39600 20400


In [2]:
model = copy.deepcopy(BaselineModel).to(device)
criterion = BaselineLoss()
optimizer = Optimizer(model.parameters(), lr=LR)

In [4]:
target_epsilon = 1.0
privacy_engine = PrivacyEngine(accountant="fed_rdp", n_clients=1)
model, optimizer, train_dataloader = privacy_engine.make_private_with_epsilon(
    module=model,
    optimizer=optimizer,
    data_loader=train_dataloader,
    epochs=NUM_STEPS,
    target_epsilon=target_epsilon,
    target_delta=TARGET_DELTA,
    max_grad_norm=MAX_GRAD_NORM,
)
if self.max_physical_batch_size > 0:
    with BatchMemoryManager(data_loader=train_dataloader,
            max_physical_batch_size=self.max_physical_batch_size,
            optimizer=optimizer) as memory_safe_dl:
        train_dataloader = memory_safe_dl
print(f"Using sigma={optimizer.noise_multiplier} and C={MAX_GRAD_NORM}")

AttributeError: 'FedRDPAccountant' object has no attribute 'sample_rate'

In [ ]:
def _standardize(M):
    '''Compute the mean of every dimension of the whole dataset'''
    [n, m] = M.shape
    if m == 1:
        print(m==1)
        return M, np.zeros(n)
    mean = np.dot(M,np.ones((m,1), dtype=np.float32)) / m
    return M - mean, mean.flatten()

def _eigen_by_lanczos(mat):
    T, V = Lanczos(mat, self.lanczos_iter)
    T_evals, T_evecs = np.linalg.eig(T)
    idx = T_evals.argsort()[-1 : -(self.proj_dims+1) : -1]
    Vk = np.dot(V.T, T_evecs[:,idx])
    return Vk

def svc(updates):
    num_weights = len(updates)
    shape_weights = [var.shape for var in updates]
    print(num_weights, shape_weights)

    for i in range(num_weights):
        _updates_normlized, mean = _standardize(updates[i].T)
        Vk = _eigen_by_lanczos(updates[i].T)

In [ ]:
train_loader_iter = iter(train_dataloader)

_previous_state = None
test_accs, i = [], 0
while i < NUM_STEPS:
    model.train()
    try:
        data, target = next(train_loader_iter)
    except StopIteration:
        train_loader_iter = iter(train_dataloader)
        data, target = next(train_loader_iter)
    data, target = data.to(device), target.to(device)
    optimizer.zero_grad()
    output = model(data)
    loss = criterion(input=output, target=target)
    loss.backward()
    optimizer.step()
    
    if len(privacy_engine.accountant) and (i == privacy_engine.accountant.history[-1][-1] - 1):
        if i % 10 == 0:
            if _previous_state is not None:
                _next_state = _model._get_current_params()
                updates = [
                    new - old for new, old in zip(_next_state, _previous_state)
                ]
            _previous_state = model._get_current_params()
            
        i += 1
        total_correct, total_points = 0, 0
        with torch.no_grad():
            model.eval()
            for data, target in iter(test_dataloader):
                data, target = data.to(device), target.to(device)
                output = model(data)

                test_correct = metric(y_true=target.detach().cpu().numpy(), y_pred=output.detach().cpu().numpy())
                total_correct += test_correct
                total_points += len(target)

            test_accs.append(round(total_correct/total_points, 4))
            print(f"Step {i}: test_correct = {total_correct}/{total_points}, " \
                  f"test_acc = {round(total_correct/total_points, 4)}")